In [4]:
import pydicom
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
import nibabel as nib
from scipy.ndimage import affine_transform
from scipy.ndimage.interpolation import rotate
from tqdm import tqdm_notebook as tqdm

plt.set_cmap('gray')
plt.rcParams['figure.figsize'] = (12, 10)

def preprocess_img(img_vol, pad=False):
    img_vol = np.rot90(img_vol.transpose(2, 0, 1), axes=(1, 2), k=3)
    img_vol = img_vol / img_vol.mean()
    img_vol = np.interp(img_vol, (img_vol.min(), img_vol.max()), (0, 1))
    img_vol = img_vol[13:-14]
    if pad:
        img_vol = np.pad(img_vol, pad_width=[(0, 0), (8, 8), (8, 8)], mode='constant', constant_values=0)
    return img_vol

def process_sag(img_vol):
    img_vol = np.rot90(img_vol, axes=(1, 2), k=1)[56:-56]
    img_vol = np.pad(img_vol, pad_width=[(0, 0), (51, 50), (8, 8)], mode='constant', constant_values=0)
    img_vol = img_vol / img_vol.mean()
    img_vol = np.interp(img_vol, (img_vol.min(), img_vol.max()), (0, 1))
    return img_vol

def gen_random_affn_mtx():
    rfn = np.random.uniform
    aff_mtx_lambdas = [
        [lambda x: rfn(0.992, 0.999), lambda x: rfn(-0.04, 0.08), lambda x: rfn(-0.04, 0.08), lambda x: rfn(-0.1, 0.2)],
        [lambda x: rfn(-0.04, 0.08), lambda x: rfn(0.992, 0.999), lambda x: rfn(-0.04, 0.08), lambda x: rfn(-2, 2)],
        [lambda x: rfn(-0.04, 0.08), lambda x: rfn(-0.04, 0.08), lambda x: rfn(0.992, 0.999), lambda x: rfn(-2, 2)]
    ]
    
    aff_mtx = np.zeros((3, 4))
    for i in range(3):
        for j in range(4):
            aff_mtx[i, j] = aff_mtx_lambdas[i][j](0)
    return aff_mtx

<Figure size 864x720 with 0 Axes>

In [7]:
# base_paths = ['/home/srivathsa/projects/brats_2021/data/train', '/home/srivathsa/projects/brats_2021/data/val']
base_paths = ['/home/srivathsa/projects/brats_2021/data/val']
# save_paths = ['/home/srivathsa/projects/studies/gad/vmorph/data/brats_t2/train', '/home/srivathsa/projects/studies/gad/vmorph/data/brats_t2/val']
save_paths = ['/home/srivathsa/projects/studies/gad/vmorph/data/brats_self/val']
for idx, base_path in enumerate(base_paths):
    case_nums = sorted([f.split('/')[-1] for f in glob('{}/*'.format(base_path))])
    save_path = save_paths[idx]

    for case_num in tqdm(case_nums, total=len(case_nums)):
        fixed = nib.load('{}/{}/{}_t1.nii.gz'.format(base_path, case_num, case_num)).get_fdata()
#         moving = nib.load('{}/{}/{}_t1ce.nii.gz'.format(base_path, case_num, case_num)).get_fdata()
        
#         fixed = process_sag(fixed)
#         moving = process_sag(moving)
        fixed = preprocess_img(fixed, pad=True)
        moving = fixed.copy()
#         moving = preprocess_img(moving, pad=True)

        aff_mtx = gen_random_affn_mtx()
        moving_tfm = affine_transform(moving, aff_mtx)
        data = np.array([fixed, moving_tfm])
        
        np.savez('{}/{}.npz'.format(save_path, case_num), data=data, aff_mtx=aff_mtx)

/home/srivathsa/miniconda3/envs/py35gad/lib/python3.6/site-packages/ipykernel_launcher.py:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if __name__ == '__main__':


  0%|          | 0/125 [00:00<?, ?it/s]

In [ ]:
base_path = '/home/srivathsa/projects/studies/gad/vmorph/data/brats/val'
dest_path = '/home/srivathsa/projects/studies/gad/vmorph/data/brats_256/val'

fpaths = sorted([f for f in glob('{}/*.npz'.format(base_path))])
for fpath in tqdm(fpaths, total=len(fpaths)):
    case_num = fpath.split('/')[-1].replace('.npz', '')
    npz = np.load(fpath)
    data = npz['data']
    aff_mtx = npz['aff_mtx']
    
    data = np.pad(data, pad_width=[(0, 0), (0, 0), (8, 8), (8, 8)], mode='constant', constant_values=0)
    np.savez('{}/{}.npz'.format(dest_path, case_num), data=data, aff_mtx=aff_mtx)

In [ ]:
plt.imshow(moving_tfm[64] - fixed[64])

In [ ]:
plt.imshow(t1_post[14] - t1_pre[14])

In [ ]:
t1_pre = nib.load('/home/srivathsa/projects/brats_2021/data/train/BraTS2021_00132/BraTS2021_00132_t1.nii.gz').get_data()
t1_post = nib.load('/home/srivathsa/projects/brats_2021/data/train/BraTS2021_00132/BraTS2021_00132_t1ce.nii.gz').get_data()

print(t1_pre.shape, t1_post.shape)

In [ ]:
plt.imshow(t1_post[..., 70] - t1_pre[..., 70])

In [ ]:
data = np.load('/home/srivathsa/projects/studies/gad/vmorph/affine_data/NO1.npz')['data']
print(data.shape)

In [ ]:
base_path = '/home/srivathsa/projects/studies/gad/vmorph/data/brats/val'
out_file = '/home/srivathsa/projects/studies/gad/vmorph/image_lists/brats/val.txt'

fpaths = sorted([f for f in glob('{}/*.npz'.format(base_path))])
fpath_str = '\n'.join(fpaths)

with open(out_file, 'w') as f:
    f.write(fpath_str)

In [ ]:
t1, t1ce = np.load('/home/srivathsa/projects/studies/gad/vmorph/brats/data/train/BraTS2021_00000.npz')['data']
print(t1.shape)

In [ ]:
plt.imshow(t1ce[64] - t1[64])

In [ ]:
base_path = '/home/srivathsa/projects/brats_2021/data/train'
case_num = 'BraTS2021_00000'
fixed = nib.load('{}/{}/{}_t1.nii.gz'.format(base_path, case_num, case_num)).get_fdata()
moving = nib.load('{}/{}/{}_t1ce.nii.gz'.format(base_path, case_num, case_num)).get_fdata()
fixed = preprocess_img(fixed)
moving = preprocess_img(moving)

In [ ]:
plt.imshow(moving[64] - t1ce[64])